#**Gender Detection Using OpenCV**

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

##**Problem Statement**


The **Adience** dataset includes a collection of Images & is intended to be as accurate as possible to the challenges of real-world imaging conditions. 

This data set was used in the paper Age and Gender Estimation of Unfiltered Faces. The paper describes the process of collecting the data set and provides additional information on the test protocols used with it.

In this demo, your goal is to predict the **Gender** of a person in Real-time using OpenCV.

##**Tasks to be Performed**

In this tutorial you will be performing the following tasks:
- Import Required Libraries
- Image Pre-processing
- Implement the Pre-trained Models
- Predict the Gender in Real-time

##**Dataset Description**



For this python project, we are going to use the **Adience** dataset.
This dataset serves as a benchmark for face photos and includes various real-world imaging conditions like noise, lighting, pose, and appearance. The sources of the images included in our set are Flickr albums, assembled by automatic upload from iPhone5 (or later) smart-phone devices, and released by their authors to the general public under the Creative Commons (CC) license.


It has a total of **26,580** photos of 2,284 subjects and is about 1GB in size. 
The models that we are going to use are trained on this dataset.

If you want to download the Dataset, [Click Here](https://www.kaggle.com/ttungl/adience-benchmark-gender-and-age-classification)

##**Skills Gained**

- OpenCV
- Deep Neural Network


##**Import Required Libraries**

In [0]:
# Please run the following code cells in your local system or Jupyter Notebook as Colab won't be able to access your PC's Webcam

import cv2 # Impoorting OpenCV Library
import math
import argparse

##**Image Pre-processing using OpenCV's Deep Neural Network Module**

OpenCV provides two functions under the Deep Neural Network Module to facilitate Image Pre-processing for Deep Learning Classification.

- cv2.dnn.blobFromImage
- cv2.dnn.blobFromImages



In [0]:
def highlightFace(net, frame, conf_threshold=0.7):
    frameOpencvDnn=frame.copy()
    frameHeight=frameOpencvDnn.shape[0]
    frameWidth=frameOpencvDnn.shape[1]
    blob=cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False) 
    
    # [blobFromImage] creates 4-dimensional blob from image. 
    # Optionally resizes and crops image from center, subtract mean values, scales values by scalefactor, swap Blue and Red channels.


    # set the input to the pre-trained deep learning network and obtain the output predicted probabilities 

    net.setInput(blob) #Passing the blob through the network 
    detections=net.forward() #Grabbing the Detections/Predictions
    faceBoxes=[]

    # Filter out weak detections by ensuring the confidence is greater than the minimum confidence
    for i in range(detections.shape[2]):
        confidence=detections[0,0,i,2]
        if confidence>conf_threshold:
            x1=int(detections[0,0,i,3]*frameWidth)
            y1=int(detections[0,0,i,4]*frameHeight)
            x2=int(detections[0,0,i,5]*frameWidth)
            y2=int(detections[0,0,i,6]*frameHeight)

            faceBoxes.append([x1,y1,x2,y2]) #Bounding Box Co-ordinates

            cv2.rectangle(frameOpencvDnn, (x1,y1), (x2,y2), (0,255,0), int(round(frameHeight/150)), 8)
    return frameOpencvDnn,faceBoxes

    #cv2.dnn.blobFromImage function returns a blob which is the input image after mean subtraction, normalizing, and channel swapping.

If you want to learn more about OpenCV's **Deep Neural Network Module** [Click Here](https://docs.opencv.org/trunk/d6/d0f/group__dnn.html#ga33d1b39b53a891e98a654fdeabba22eb)

##**Implement the DNN based Pre-trained Model**

In [0]:
parser=argparse.ArgumentParser()
parser.add_argument('--image')

args=parser.parse_args()

# The following two files are protobuf file (protocol buffer).
# They contains the graph definition and the trained weights of the model. 
# A .pb file contains the protobuf file in binary format, the .pbtxt extension contains it in text format. 

faceProto="opencv_face_detector.pbtxt"
faceModel="opencv_face_detector_uint8.pb"

genderProto="gender_deploy.prototxt" # Describes the Network Configuration
genderModel="gender_net.caffemodel" # Defines the Internal States of the parameters of the Layers

MODEL_MEAN_VALUES=(78.4263377603, 87.7689143744, 114.895847746) #Initializing the Mean Values for the Model
genderList=['Male','Female']

#readNet() method is used to load the Networks

#First parameter contains Trained Weights
#Second parameter contains Network Configuration

faceNet=cv2.dnn.readNet(faceModel,faceProto) 
genderNet=cv2.dnn.readNet(genderModel,genderProto)


##**Capture the Video in Real-time & predict the Gender**

In [0]:
video=cv2.VideoCapture(args.image if args.image else 0)
padding=20

while cv2.waitKey(1)<0:
    hasFrame,frame=video.read()
    if not hasFrame:
        cv2.waitKey()
        break

    resultImg,faceBoxes=highlightFace(faceNet,frame)
    if not faceBoxes:
        print()

    for faceBox in faceBoxes:
        face=frame[max(0,faceBox[1]-padding):
                   min(faceBox[3]+padding,frame.shape[0]-1),max(0,faceBox[0]-padding)
                   :min(faceBox[2]+padding, frame.shape[1]-1)]

        blob=cv2.dnn.blobFromImage(face, 1.0, (227,227), MODEL_MEAN_VALUES, swapRB=False) 
        #returns a blob which is the input image after mean subtraction, normalizing, and channel swapping.
        

        # Making predictions on the Gender and find the Gender bucket with the largest corresponding probability

        genderNet.setInput(blob) #Passing the blob through the Neural Net
        genderPreds=genderNet.forward() #Grabbing the Detections/Predictions
        gender=genderList[genderPreds[0].argmax()] #Displaying the Top Prediction
        #print(f'Gender: {gender}')

        cv2.putText(resultImg, f'{gender}', (faceBox[0], faceBox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 2, cv2.LINE_AA)
        #The function putText renders the specified text string in the image.
        
        #If you want to learn more about OpenCV Drawing Functions visit - https://docs.opencv.org/2.4/modules/core/doc/drawing_functions.html
        
        cv2.imshow("Gender Detection", resultImg) #Display the Image in an OpenCV Window
